In [23]:
import urllib.request
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import json
import re
import datetime
from tqdm import tqdm
import requests
import time
from requests.adapters import HTTPAdapter, Retry
import pickle
import os

# Download all webarchive pages

In [5]:
night_tours_dates_url = "https://web.archive.org/cdx/search/cdx?url=https://www.nighttours.com/gaypride/&output=json"

# night_tours_dates_resource = urllib.request.urlopen(night_tours_dates_url, timeout=20)
# night_tours_dates_content =  night_tours_dates_resource.read().decode(
#     night_tours_dates_resource.headers.get_content_charset())
# night_tours_dates = json.loads(night_tours_dates_content)
night_tours_dates_resource = requests.get(night_tours_dates_url)

In [6]:
night_tours_dates_content =  night_tours_dates_resource.content.decode(
    night_tours_dates_resource.encoding)
night_tours_dates = json.loads(night_tours_dates_content)

In [7]:
timestamp_col = night_tours_dates[0].index('timestamp')
night_tours_dates = [snapshot[timestamp_col] for snapshot in night_tours_dates[1:]]
night_tours_dates = sorted(list(set(night_tours_dates)))

In [8]:
soups=[]

In [10]:
# for ts in [night_tours_dates[0], 
#            night_tours_dates[len(night_tours_dates)//3], 
#            night_tours_dates[2 * len(night_tours_dates)//3],
#            night_tours_dates[-1]]:
headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    "Accept-Encoding": "gzip, deflate, br",
    'Accept-language': 'he-IL,he;q=0.9,en-US;q=0.8,en;q=0.7,fr;q=0.6',
#     'Content-Type': 'application/json',
    "Connection": "keep-alive",
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',    
}

session = requests.Session()
retries = Retry(total=5,
                backoff_factor=0.5,
                status_forcelist=[ 500, 502, 503, 504 ],
                connect=3)

# retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retries)
session.mount('http://', adapter)
session.mount('https://', adapter)

for ts in tqdm(night_tours_dates[len(soups):]):
#     resource = urllib.request.urlopen(f"https://web.archive.org/web/{ts}/https://www.nighttours.com/gaypride/")
    resource = session.get(f"https://web.archive.org/web/{ts}/http://www.nighttours.com/gaypride/",
                            timeout=120, headers=headers)
#     data =  resource.read().decode(resource.headers.get_content_charset())
    
    with open(f'gayparades_htmls\\{ts}_{resource.encoding}_{resource.apparent_encoding}.html', "wb") as f:
        f.write(resource.content)

    time.sleep(25)

100%|██████████████████████████████████████████████████████████████████████████████| 268/268 [2:02:55<00:00, 27.52s/it]


# Load all html pages

In [10]:
soups = []
for path in sorted(os.listdir('gayparades_htmls')):
    html = os.path.join('gayparades_htmls', path)
    with open(html, 'rb') as f:
        resource = f.read()
    
    ts, encoding, apparent_encoding = path.split('.')[0].split('_')
    
    try:
        data = resource.decode(encoding)
    except UnicodeDecodeError:
        data = resource.decode(apparent_encoding)
    soup = BeautifulSoup(data, 'html.parser')
    soups.append((soup, ts))

In [26]:
skipped_soups = []
dates = []
for s, ts in soups:
    pridelist = s.find("div", {"id": "pridelist"})
    if pridelist is None:
        print(f'Skipping {ts}: no pridelist')
        skipped_soups.append((s,ts))
        continue
        
    events = pridelist.find_all("div", {"itemtype": "http://schema.org/Event"})
    if not len(events):
        print(f'Skipping {ts}: no event div')
        skipped_soups.append((s,ts))
        continue

    for e in events:
        if 'cancelled' in e.get_text():
                    continue
                
        if e.find('span', {'class': 'flag_us'}) is not None:
            start_date = e.find('time', {'itemprop': 'startDate'}).attrs['datetime']
            end_date = e.find('time', {'itemprop': 'endDate'})
            if end_date is None:
                end_date = start_date
            else:
                end_date = end_date.attrs['datetime']
            start_date = datetime.date.fromisoformat(start_date)
            end_date = datetime.date.fromisoformat(end_date)
            location = e.find('span', {'itemprop': 'addressLocality'}).get_text()
            description = e.find('span', {'itemprop': 'name'}).get_text()
            dates.append({
                'location': location,
                'description': description,
                'start_date': start_date,
                'end_date': end_date,
                'flagged': True, 'ts': ts
            })

print(f'Processed {len(soups) - len(skipped_soups)}/{len(soups)} htmls.\n So far {len(dates)} dates.')

Skipping 20061026221710: no pridelist
Skipping 20061208225851: no pridelist
Skipping 20061213175746: no pridelist
Skipping 20061218102202: no pridelist
Skipping 20061223013001: no pridelist
Skipping 20061231095217: no pridelist
Skipping 20070105015532: no pridelist
Skipping 20070107032131: no pridelist
Skipping 20070109205651: no pridelist
Skipping 20070114195410: no pridelist
Skipping 20070119213526: no pridelist
Skipping 20070125091636: no pridelist
Skipping 20070206002150: no pridelist
Skipping 20070307091905: no pridelist
Skipping 20070316043702: no pridelist
Skipping 20070324050528: no pridelist
Skipping 20070509150816: no pridelist
Skipping 20070518172714: no pridelist
Skipping 20070625174903: no pridelist
Skipping 20070627061632: no pridelist
Skipping 20070717074437: no pridelist
Skipping 20070810140931: no pridelist
Skipping 20070823150350: no pridelist
Skipping 20070925160559: no pridelist
Skipping 20071008044920: no pridelist
Skipping 20071011072533: no pridelist
Skipping 200

In [27]:
MONTHS_NAMES = {
    'jan': 1,
    'gen': 1,
    'feb': 2,
    'fév': 2,
    'mar': 3,
    'apr': 4,
    'avr': 4,
    'may': 5,
    'mag': 5,
    'mai': 5,
    'jun': 6,
    'giu': 6,
    'jul': 7,
    'lug': 7,
    'aug': 8,
    'ago': 8,
    'aou': 8,
    'sep': 9,
    'set': 9,
    'oct': 10,
    'ott': 10,
    'nov': 11,
    'dec': 12,
    'déc': 12,
    'dic': 12,
}

WEEKDAYS = '(Sun|Mon|Tue|Wed|Thu|Fri|Sat)'
MONTHS = '(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|' \
          'gen|feb|mar|apr|mag|giu|lug|ago|set|ott|nov|dic|' \
          'Fév|Avr|Mai|Aou|Déc)'

skipped_soups2 = []
for s, ts in skipped_soups:
    pridelist = s.find("div", {"id": "pridelist"})
    if pridelist is None:
        maincontent = s.find("div", {"id": "maincontent"})
        if maincontent is None:
            print(f'Skipping {ts}: no pridelist, no maincontent')
            skipped_soups2.append((s,ts))
            continue
            
        maincontent_txt = maincontent.get_text().replace(u'\xa0', u' ').replace('\u2009', u' ')       
        matches = re.findall(f'\s*(.+?)\n'
                             f'((\d+)\s+?[-–]\s+?(\d+)\s+?{MONTHS}|'
                             f'(\d+)\s+?{MONTHS}\s+?[-–]\s+?(\d+)\s+?{MONTHS}|'
                             f'(\d+)\s+?{MONTHS})\s*?(\d\d\d\d)?\n(.+?)\n', 
                             maincontent_txt)
        if not len(matches):
            print(f'Skipping {ts}: regex matches')
            skipped_soups2.append((s,ts))
            continue
        
        dates += [{'location': m[0], 'description': m[-1],
                   'start_date': datetime.date(int(m[-2] if len(m[-2]) else ts[:4]),
                                               MONTHS_NAMES[(m[4]+m[6]+m[10]).lower()],
                                               int(m[2]+m[5]+m[9])),
                   'end_date': datetime.date(int(m[-2] if len(m[-2]) else ts[:4]),
                                             MONTHS_NAMES[(m[4]+m[8]+m[10]).lower()],
                                             int(m[3]+m[7]+m[9])),
                  'flagged': False, 'ts': ts} for m in matches]
    else:
        # pridelist but no events
        events = pridelist.find_all("li", {"itemtype": "http://schema.org/Event"})
        if not len(events):
            print(f'Skipping {ts}: no event li')
            skipped_soups2.append((s,ts))
            continue
            
        for e in events:
            if 'cancelled' in e.get_text():
                    continue
            
            if e.find('span', {'class': 'flag_us'}) is not None:
                start_date = e.find('time', {'itemprop': 'startDate'}).attrs['datetime']
                end_date = e.find('time', {'itemprop': 'endDate'})
                if end_date is None:
                    end_date = start_date
                else:
                    end_date = end_date.attrs['datetime']
                start_date = datetime.date.fromisoformat(start_date)
                end_date = datetime.date.fromisoformat(end_date)
                location = e.find('span', {'itemprop': 'addressLocality'}).get_text()
                description = e.find('span', {'itemprop': 'name'}).get_text()
                dates.append({
                    'location': location,
                    'description': description,
                    'start_date': start_date,
                    'end_date': end_date,
                    'flagged': True, 'ts': ts
                })

print(f'Processed {len(soups) - len(skipped_soups2)}/{len(soups)} htmls.\n So far {len(dates)} dates.')

Skipping 20070316043702: no pridelist, no maincontent
Skipping 20070324050528: no pridelist, no maincontent
Skipping 20070509150816: no pridelist, no maincontent
Skipping 20070518172714: no pridelist, no maincontent
Skipping 20070625174903: no pridelist, no maincontent
Skipping 20070627061632: no pridelist, no maincontent
Skipping 20070717074437: no pridelist, no maincontent
Skipping 20070810140931: no pridelist, no maincontent
Skipping 20070823150350: no pridelist, no maincontent
Skipping 20070925160559: no pridelist, no maincontent
Skipping 20071008044920: no pridelist, no maincontent
Skipping 20071011072533: no pridelist, no maincontent
Skipping 20071018040432: no pridelist, no maincontent
Skipping 20071117111838: no pridelist, no maincontent
Skipping 20071217191524: no pridelist, no maincontent
Skipping 20080117051103: no pridelist, no maincontent
Skipping 20080217232117: no pridelist, no maincontent
Skipping 20080319103315: no pridelist, no maincontent
Skipping 20080419012735: no 

In [28]:

skipped_soups3 = []
for s, ts in skipped_soups2:
    pridelist = s.find("div", {"id": "pridelist"})
    if pridelist is None:
        maincontent = s.find("div", {"id": "maincontent"})
        if maincontent is None:
            leftcolumn = s.find("div", {"id": "leftcolumn"})
            if leftcolumn is None:
                print(f'Skipping {ts}: no pridelist, no maincontent, no leftcolumn')
                skipped_soups3.append((s,ts))
                continue
            
            events = leftcolumn.find_all('div', {'class': 'rowoff'})
            if not len(events):
                print(f'Skipping {ts}: no rowoff div')
                skipped_soups3.append((s,ts))
                continue
            
            
            if events[0].find('div', {'class': 'column_desc'}) is None or events[0].find_all('div', {'class': 'column_dateplace'}) is None:
                    print(f'Skipping {ts}: no column_desc or date_place div')
                    skipped_soups3.append((s,ts))
                    continue
            
            for e in events:
                if 'cancelled' in e.get_text():
                    continue
                
                # try to find flag
                if e.find('img') is not None:
                    if 'us' not in e.find('img').attrs['alt'].lower():
                        continue
                
                desc = e.find('div', {'class': 'column_desc'}).get_text()
                loc, date = e.find_all('div', {'class': 'column_dateplace'})
                
                m = re.findall('((\d+)\s+?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)|'
                           '(\d+)\s+?[-–]\s+?(\d+) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)|'
                           '(\d+)\s+?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+?[-–]\s+?'
                           '(\d+)\s+?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))(\s+?(20\d\d))?',
                               date.get_text().replace(u'\xa0', u' ').replace('\u2009', u' '))[0]
                
                dates.append({'location': loc.get_text(),
                              'description': desc,
                              'start_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                  MONTHS_NAMES[(m[2]+m[5]+m[7]).lower()], int(m[1]+m[3]+m[6])),
                              'end_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                 MONTHS_NAMES[(m[2]+m[5]+m[9]).lower()], int(m[1]+m[4]+m[8])),
                              'flagged': False, 'ts': ts
                             })
        else:
            maincontent_txt = maincontent.get_text()
            matches = re.findall('\s*(.+?)\n((\d+)\s+?'
                                 '(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)|'
                                 '(\d+)\s+?[-–]\s+?(\d+) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)|'
                                 '(\d+)\s+?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+?'
                                 '[-–]\s+?(\d+) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))(\s+?'
                                 '(20\d\d))?\n(.+?)\n', maincontent_txt)
            if not len(matches):
                print(f'Skipping {ts}: regex matches2')
                skipped_soups3.append((s,ts))
                continue
        
            dates += [{'location': m[0],
                       'description': m[-1],
                       'start_date': datetime.date(int(m[-2] if len(m[-2]) else ts[:4]), 
                                                   MONTHS_NAMES[(m[3]+m[6]+m[8]).lower()], int(m[2]+m[4]+m[7])),
                       'end_date': datetime.date(int(m[-2] if len(m[-2]) else ts[:4]), 
                                                 MONTHS_NAMES[(m[3]+m[6]+m[10]).lower()], int(m[2]+m[5]+m[9])),
                       'flagged': False, 'ts': ts
                       } for m in matches]
    else:
        # pridelist but no events
        events = pridelist.find_all("li")
        if not len(events):
            print(f'Skipping {ts}: no event li at all')
            skipped_soups3.append((s,ts))
            continue
        
        if pridelist.find('span', {'class': 'flag_us'}) is None and pridelist.find('img', {'alt': 'flag USA'}) is None \
            and pridelist.find('span', {'class': 'flag-us'}) is None:
            gaegaeg
            print(f'Skipping {ts}: no flags?')
            skipped_soups3.append((s,ts))
            continue
            
        try:
            for e in events:
                if 'cancelled' in e.get_text():
                    continue
                
                if e.find('span', {'class': 'flag_us'}) is not None or e.find('img', {'alt': 'flag USA'}) is not None or \
                    e.find('span', {'class': 'flag-us'}) is not None:
                    date = e.find('span', {'class': 'cel-date'})
                    if date is not None:
                        m = re.findall(f'((\d+)\s+?[-–]\s+?(\d+)\s+?{MONTHS}|'
                                       f'(\d+)\s+?{MONTHS}\s+?[-–]\s+?(\d+)\s+?{MONTHS}|'
                                       f'(\d+)\s+?{MONTHS})(\s+?(20\d\d))?',
                                       date.get_text().replace(u'\xa0', u' ').replace('\u2009', u' '))
                        
                        if len(m):
                            m = m[0]
                        else:
                            raise NotImplementedError("matches date")
                        
                        city = e.find('span', {'class': 'cel-city-event'})
                        if city is None:
                            city = e.find('span', {'class': 'cel-info-after-arrow'})

                        dates.append({'location': city.contents[-1].get_text(),
                          'description': (str(city.contents[0]) if type(city.contents[0]) == NavigableString else
                                          city.contents[0].get_text()),
                          'start_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                      MONTHS_NAMES[(m[3]+m[5]+m[9]).lower()],
                                                      int(m[1]+m[4]+m[8])),
                          'end_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                    MONTHS_NAMES[(m[3]+m[7]+m[9]).lower()],
                                                    int(m[2]+m[6]+m[8])),
                          'flagged': True, 'ts': ts
                         })
                    else: 
                        desc = e.find('span', {'class': 'cel-info'}).contents[0].get_text()
                        citydate = e.find('span', {'class': 'cel-city'})

                        m = re.findall(f'\s*(.+?)\s+?–\s+?('
                                       f'({WEEKDAYS}\s+?)?(\d+)\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                                       f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                                       f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS})(\s+(20\d\d))?', 
                                       citydate.get_text().replace(u'\xa0', u' ').replace('\u2009', u' '))
                        
                        if len(m):
                            m = m[0]
                        else:
                            if 'TBA' in citydate.get_text():
                                continue
                            else:
                                afefeaf
                                raise NotImplementedError("matches")

                        dates.append({'location': m[0],
                                      'description': desc,
                                      'start_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                                  MONTHS_NAMES[(m[8]+m[12]+m[20]).lower()], 
                                                                  int(m[4]+m[11]+m[19])),
                                      'end_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                                MONTHS_NAMES[(m[8]+m[16]+m[20]).lower()], 
                                                                int(m[7]+m[15]+m[19])),
                                      'flagged': True, 'ts': ts
                                     })
        except NotImplementedError as ee:
            print(f'Skipping {ts}: matches{ee}')
            skipped_soups3.append((s,ts))
            continue
                         


print(f'Processed {len(soups) - len(skipped_soups3)}/{len(soups)} htmls.\n So far {len(dates)} dates.')

Skipping 20081220152307: no pridelist, no maincontent, no leftcolumn
Skipping 20090121173320: no pridelist, no maincontent, no leftcolumn
Skipping 20090219140144: no pridelist, no maincontent, no leftcolumn
Skipping 20090221180808: no pridelist, no maincontent, no leftcolumn
Skipping 20090222192240: no pridelist, no maincontent, no leftcolumn
Skipping 20090421070047: no pridelist, no maincontent, no leftcolumn
Skipping 20090820180634: no pridelist, no maincontent, no leftcolumn
Skipping 20100114173435: no pridelist, no maincontent, no leftcolumn
Skipping 20100209005320: no pridelist, no maincontent, no leftcolumn
Skipping 20100424005357: no pridelist, no maincontent, no leftcolumn
Skipping 20100429143621: no pridelist, no maincontent, no leftcolumn
Skipping 20101009205817: no pridelist, no maincontent, no leftcolumn
Skipping 20101023064719: no pridelist, no maincontent, no leftcolumn
Skipping 20101129021808: no pridelist, no maincontent, no leftcolumn
Skipping 20101221204723: no pridel

In [29]:
skipped_soups4 = []
for s, ts in skipped_soups3:
    list_full_events = s.find("div", {"class": "list_full_events"})
    if list_full_events is None:
        print(f'Skipping {ts}: no pridelist, no list_full_events')
        skipped_soups4.append((s,ts))
        continue

    events = list_full_events.find_all("div", {"itemtype": "http://schema.org/Event"})
    if not len(events):
        content_basic468 = s.find("div", {"class": "content_basic468"})
        if content_basic468 is None:
            print(f'Skipping {ts}: no events')
            skipped_soups4.append((s,ts))
            continue
        
        events = list_full_events.find_all("a")
        if not len(events):
            print(f'Skipping {ts}: basic but no events')
            skipped_soups4.append((s,ts))
            continue
            
        if list_full_events.find('span', {'class': 'flag_us'}) is None:
            print(f'Skipping {ts}: basic but no flags')
            skipped_soups4.append((s,ts))
            continue
        
        for e in events:
            if e.find('span', {'class': 'flag_us'}) is not None:
                loc = e.find('span', {'class': 'cel2'}).get_text()             
                desc = e.find('span', {'class': 'cel4'}).get_text()             
                date = e.find('span', {'class': 'cel3'}).get_text().replace(u'\xa0', u' ').replace('\u2009', u' ')

                m = re.findall(f'(({WEEKDAYS}\s+?)?(\d+)\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                               f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                               f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS})(\s+(20\d\d))?', date)
                
                if len(m):
                    m = m[0]
                else:
                    if 'TBA' in citydate.get_text():
                        continue
                    else:
                        raise NotImplementedError("matches")

                dates.append({'location': loc,
                              'description': desc,
                              'start_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                          MONTHS_NAMES[(m[7]+m[11]+m[19]).lower()], 
                                                          int(m[3]+m[10]+m[18])),
                                      'end_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                                MONTHS_NAMES[(m[7]+m[15]+m[19]).lower()], 
                                                                int(m[6]+m[14]+m[18])),
                              'flagged': True, 'ts': ts
                             })

    else:
        if list_full_events.find('span', {'class': 'flag_us'}) is None:
            print(f'Skipping {ts}: no flags')
            skipped_soups4.append((s,ts))
            continue

        for e in events:
            if e.find('span', {'class': 'flag_us'}) is not None:
                loc = e.find('span', {'itemprop': 'addressLocality'}).get_text()             
                start_date = e.find('time', {'itemprop': 'startDate'}).attrs['datetime']
                end_date = e.find('time', {'itemprop': 'endDate'})
                if end_date is None:
                    end_date = start_date
                else:
                    end_date = end_date.attrs['datetime']
                start_date = datetime.date.fromisoformat(start_date)
                end_date = datetime.date.fromisoformat(end_date)
                description = e.find('span', {'itemprop': 'name'}).get_text()

                dates.append({
                    'location': location,
                    'description': description,
                    'start_date': start_date,
                    'end_date': end_date,
                    'flagged': True, 'ts': ts
                })
        
print(f'Processed {len(soups) - len(skipped_soups4)}/{len(soups)} htmls.\n So far {len(dates)} dates.')

Skipping 20081220152307: no pridelist, no list_full_events
Skipping 20090121173320: no pridelist, no list_full_events
Skipping 20090219140144: no pridelist, no list_full_events
Skipping 20090221180808: no pridelist, no list_full_events
Skipping 20090222192240: no pridelist, no list_full_events
Skipping 20090421070047: no pridelist, no list_full_events
Skipping 20090820180634: no pridelist, no list_full_events
Skipping 20100114173435: no events
Skipping 20100209005320: no events
Skipping 20100424005357: no events
Skipping 20100429143621: no events
Processed 257/268 htmls.
 So far 9378 dates.


In [30]:
skipped_soups5 = []
for s, ts in skipped_soups4:
    list_full_events = s.find("div", {"class": "list_full_events"})
    if list_full_events is None:
        midcontainer = s.find("div", {"id": "midcontainer"})
        if midcontainer.find("div", {"class": "column_flag"}) is None:
            print(f'Skipping {ts}: no flags?')
            skipped_soups5.append((s,ts))
            continue
            
        events = midcontainer.find_all('div', {'class': 'rowoff'})
        if not len(events):
            print(f'Skipping {ts}: no rowoff div')
            skipped_soups5.append((s,ts))
            continue
        
            
        if events[0].find('div', {'class': 'column_desc'}) is None or events[0].find_all('div', {'class': 'column_dateplace'}) is None:
            print(f'Skipping {ts}: no column_desc or date_place div')
            skipped_soups5.append((s,ts))
            continue
            
        for e in events:
            if 'cancelled' in e.get_text():
                continue

            # try to find flag
            if 'us' not in e.find('img').attrs['alt'].lower():
                continue

            desc = e.find('div', {'class': 'column_desc'}).get_text()
            loc, date = e.find_all('div', {'class': 'column_dateplace'})

            m = re.findall('((\d+)\s+?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)|'
                       '(\d+)\s+?[-–]\s+?(\d+) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)|'
                       '(\d+)\s+?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+?[-–]\s+?'
                       '(\d+)\s+?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))(\s+?(20\d\d))?',
                           date.get_text().replace(u'\xa0', u' ').replace('\u2009', u' '))[0]

            dates.append({'location': loc.get_text(),
                          'description': desc,
                          'start_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                              MONTHS_NAMES[(m[2]+m[5]+m[7]).lower()], int(m[1]+m[3]+m[6])),
                          'end_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                             MONTHS_NAMES[(m[2]+m[5]+m[9]).lower()], int(m[1]+m[4]+m[8])),
                          'flagged': True, 'ts': ts
                         })
        
    else:
        
        events = list_full_events.find_all("li")
        if not len(events):
            print(f'Skipping {ts}: no events')
            skipped_soups5.append((s,ts))
            continue
        
        if e.find('span', {'class': 'flag_us'}) is not None:
            loc = e.find('span', {'class': 'cel2'}).get_text()             
            desc = e.find('span', {'class': 'cel4'}).get_text()             
            date = e.find('span', {'class': 'cel3'}).get_text().replace(u'\xa0', u' ').replace('\u2009', u' ')

            m = re.findall(f'(({WEEKDAYS}\s+?)?(\d+)\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                           f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                           f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS})(\s+(20\d\d))?', date)
                
            if len(m):
                m = m[0]
            else:
                if 'TBA' in citydate.get_text():
                    continue
                else:
                    raise NotImplementedError("matches")

            dates.append({'location': loc,
                          'description': desc,
                          'start_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                      MONTHS_NAMES[(m[7]+m[11]+m[19]).lower()], 
                                                      int(m[3]+m[10]+m[18])),
                                  'end_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                            MONTHS_NAMES[(m[7]+m[15]+m[19]).lower()], 
                                                            int(m[6]+m[14]+m[18])),
                          'flagged': True, 'ts': ts
                         })
        
        
print(f'Processed {len(soups) - len(skipped_soups5)}/{len(soups)} htmls.\n So far {len(dates)} dates.')

Skipping 20090219140144: no flags?
Skipping 20090221180808: no flags?
Skipping 20090222192240: no flags?
Skipping 20090421070047: no flags?
Skipping 20090820180634: no flags?
Skipping 20100424005357: no events
Skipping 20100429143621: no events
Processed 261/268 htmls.
 So far 9407 dates.


In [31]:
skipped_soups6 = []
for s, ts in skipped_soups5:
    list_full_events = s.find("div", {"class": "list_full_events"})
    if list_full_events is None:
        midcontainer = s.find("div", {"id": "midcontainer"})
        pridelistfull = s.find("div", {"class": "pridelistfull"})
        if pridelistfull is None:
            print(f'Skipping {ts}: no pridelistfull')
            skipped_soups6.append((s,ts))
            continue
            
        events = midcontainer.find_all('div', {'class': 'row'})
        if not len(events):
            print(f'Skipping {ts}: no row div')
            skipped_soups6.append((s,ts))
            continue
        
        if events[0].find('div', {'class': 'cel1'}) is None:
            print(f'Skipping {ts}: wrong format')
            skipped_soups6.append((s,ts))
            continue
            
        for e in events:
            if 'cancelled' in e.get_text():
                continue

            # try to find flag
            if 'us' not in e.find('img').attrs['alt'].lower():
                continue
            loc = e.find('div', {'class': 'cel2'}).get_text()             
            desc = e.find('div', {'class': 'cel4'}).get_text()             
            date = e.find('div', {'class': 'cel3'}).get_text().replace(u'\xa0', u' ').replace('\u2009', u' ')

            m = re.findall(f'(({WEEKDAYS}\s+?)?(\d+)\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                           f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                           f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS})(\s+(20\d\d))?', date)
                
            if len(m):
                m = m[0]
            else:
                if 'TBA' in citydate.get_text():
                    continue
                else:
                    raise NotImplementedError("matches")

            dates.append({'location': loc,
                          'description': desc,
                          'start_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                      MONTHS_NAMES[(m[7]+m[11]+m[19]).lower()], 
                                                      int(m[3]+m[10]+m[18])),
                          'end_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                            MONTHS_NAMES[(m[7]+m[15]+m[19]).lower()], 
                                                            int(m[6]+m[14]+m[18])),
                          'flagged': True, 'ts': ts
                         })
        
    else:
        events = list_full_events.find_all("a")
        if not len(events):
            print(f'Skipping {ts}: no events again')
            skipped_soups6.append((s,ts))
            continue
            
        if list_full_events.find('span', {'class': 'flag_us'}) is None:
            print(f'Skipping {ts}: no flags again')
            skipped_soups6.append((s,ts))
            continue
        
        for e in events:
            if e.find('span', {'class': 'flag_us'}) is not None:
                loc = e.find('span', {'class': 'cel2'}).get_text()             
                desc = e.find('span', {'class': 'cel4'}).get_text()             
                date = e.find('span', {'class': 'cel3'}).get_text().replace(u'\xa0', u' ').replace('\u2009', u' ')

                m = re.findall(f'(({WEEKDAYS}\s+?)?(\d+)\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                               f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}\s+?[-–]\s+?({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS}|'
                               f'({WEEKDAYS}\s+?)?(\d+)\s+?{MONTHS})(\s+(20\d\d))?', date)
                
                if len(m):
                    m = m[0]
                else:
                    if 'TBA' in citydate.get_text():
                        continue
                    else:
                        raise NotImplementedError("matches")

                dates.append({'location': loc,
                              'description': desc,
                              'start_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                          MONTHS_NAMES[(m[7]+m[11]+m[19]).lower()], 
                                                          int(m[3]+m[10]+m[18])),
                              'end_date': datetime.date(int(m[-1] if len(m[-1]) else ts[:4]), 
                                                                MONTHS_NAMES[(m[7]+m[15]+m[19]).lower()], 
                                                                int(m[6]+m[14]+m[18])),
                              'flagged': True, 'ts': ts
                             })
        
        
print(f'Processed {len(soups) - len(skipped_soups6)}/{len(soups)} htmls.\n So far {len(dates)} dates.')

Processed 268/268 htmls.
 So far 9544 dates.


In [41]:
sorted_dates = sorted(dates, key=lambda d: d['start_date'])

In [42]:
filt_sorted_dates = {} 

for d in list(sorted_dates):
    filt_sorted_dates[(d['location'], d['start_date'], d['end_date'])] = {'desc': d['description'],
                                                                          'flagged': d['flagged'],
                                                                          'ts': d['ts']}

# write to csv

In [50]:
len( filt_sorted_dates.keys())

950

In [46]:
len([k for k in filt_sorted_dates.keys() if filt_sorted_dates[k]['flagged']])

874

In [43]:
import csv

with open('unfiltered.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['location', 'start_date', 'end_date', 'description', 'USA for sure', 'timestamp']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for d in sorted_dates:
        writer.writerow({'location': d['location'],
                         'start_date': d['start_date'],
                         'end_date': d['end_date'],
                         'description': d['description'],
                         'USA for sure': d['flagged'],
                         'timestamp': d['ts'],
                        })
})

In [49]:
import csv
with open('filtered.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['location', 'start_date', 'end_date', 'description', 'USA for sure', 'timestamp']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for k in filt_sorted_dates:
        writer.writerow({'location': k[0],
                         'start_date': k[1],
                         'end_date': k[2],
                         'description': filt_sorted_dates[k]['desc'],
                         'USA for sure': filt_sorted_dates[k]['flagged'],
                         'timestamp': filt_sorted_dates[k]['ts'],
                        })